In [1]:
# Import necessary libraries and modules
from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import base64
import os
import numpy as np
import pandas as pd
import plotly.express as px
from project1Crud import Project1Database

# Read and encode your image
image_filename = 'Grazioso Salvare Logo.png'  # image file
with open(image_filename, 'rb') as f:
    encoded_image = base64.b64encode(f.read())

# Initialize the Dash app
app = JupyterDash(__name__)

# Connect to your database via your CRUD module
# Provide your MongoDB username and password
username = "aacuser"
password = "YourPasswordHere"
db = Project1Database(username, password)

# Read data from your database
df = pd.DataFrame.from_records(db.read({}))
df.drop(columns=['_id'], inplace=True)  

# Define your layout
app.layout = html.Div([
    html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), id='your-unique-identifier'),
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div([
        html.Label("Select a Filter:"),
        dcc.Dropdown(
            id='filter-type',
            options=[
                {'label': 'All Animals', 'value': 'all'},
                {'label': 'Dogs', 'value': 'dogs'},
                {'label': 'Cats', 'value': 'cats'}
            ],
            value='all'  # Default selected value
        ),
    ]),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        style_table={'overflowX': 'auto'},  # Enable horizontal scrolling if needed
        sort_action='native',  # Enable native sorting
        filter_action='native',  # Enable native filtering
        row_selectable='single'  # Enable single-row selection
    ),
    html.Div(
        className='row',
        style={'display': 'flex'},
        children=[
            html.Div(id='graph-id', className='col s12 m6'),
            html.Div(id='map-id', className='col s12 m6'),
        ]
    )
])

# Define your callback functions
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_data_table(selected_filter):
    print("Selected filter:", selected_filter)
    
    if selected_filter == 'all':
        return df.to_dict('records')
    elif selected_filter == 'dogs':
        filtered_data = df[df['animal_type'] == 'Dog']
        print("Filtered data (Dogs):", filtered_data)  # Add this line for debugging
        return filtered_data.to_dict('records')
    elif selected_filter == 'cats':
        filtered_data = df[df['animal_type'] == 'Cat']
        print("Filtered data (Cats):", filtered_data)  # Add this line for debugging
        return filtered_data.to_dict('records')


# Chart
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data"),
     #Input('filter-type', 'value')
    ])
def update_graphs(viewData):
    ###FIX ME ####
    # add code for chart of your choice (e.g. pie chart) #
    dff = pd.DataFrame.from_dict(viewData)
    
    return [
        dcc.Graph(            
#             figure = px.pie(dff, values='breed',  
#                             title='Available Dogs by Breed')
            figure = px.histogram(dff, x='breed')
        )    
    ]

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, index):
    if viewData is None or index is None:
        return
    
    dff = pd.DataFrame.from_dict(viewData)
    row = index[0] if index else 0
    
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


Dash app running on http://127.0.0.1:12376/
Selected filter: all
Selected filter: dogs
Filtered data (Dogs):       rec_num age_upon_outcome animal_id animal_type  \
0           8           1 year   A736551         Dog   
1           6          5 years   A696004         Dog   
2           9          3 years   A720214         Dog   
4          11           1 year   A721199         Dog   
5          12           1 year   A664843         Dog   
...       ...              ...       ...         ...   
9985     9999         2 months   A665304         Dog   
9987     9993          2 years   A761792         Dog   
9988     9991          6 years   A672341         Dog   
9989     9987          4 years   A729430         Dog   
9990     9986           1 year   A728197         Dog   

                                         breed        color date_of_birth  \
0     Labrador Retriever/Australian Cattle Dog        Black    2015-10-12   
1                     Cardigan Welsh Corgi Mix  Sable/White    2